In [39]:
library(biomaRt)
library(dplyr)
library(readr)
library(readxl)
library(VennDiagram)
library(ggplot2)
library(sf)

ERROR: Error in library(VennDiagram): there is no package called ‘VennDiagram’


In [2]:
humanlist <- c("acp2","calsyntenin-1","calsyntenin-2","calsyntenin-3","aplp1","aplp2","app","axl","bcma","betacellulin","betaglycan","car","cd147","cd200","cd43","cd44","cd99","cx3cl1","cxcl16","desmoglein-2","dner","dr6","dystroglycan","e-cadherin","epcam","epha2","epha5","epha7","ephb3","ephb4","ephb6","ephrin-b2","erbb4","f11r","fgfr4","glg1","hla-a2","ifnar2","igf-1r","il11r","il-1r1","il-1r2","il6r","ir","irela","ilrelb","jagged2","kcne2","kcne1","klotho","lar","ldlr","lrp1","lrp1b","lrp6","mer","met","muc1","musk","n-cadherin","neogenin","neurexin-3-b","NLRR3","p75NTR","pianp","plxdc2","pmel17","pdpn","polycystin-1","polyductin","protocadherin-12","ptk7","rage","robo1","sez6","sez6l","sez6l2","sirpa","sorcs1","sorla","sortilin","synedecan-1","synedecan-2","synedecan-3","tie1","tmeff2","tnfr1","trem2","trka","tyro3","vasorin","vegfr3")

In [3]:
mouselist <- c("apoER2","CACHD1","CADM1","CSF1R","Delta1","dscam","dscaml1","ephb2","ephrin-b2","fgfr3","ghr","l1","nectin1a","nectin3","ng2","notch1","notch2","notch3","notch4","nprc","nradd","pianp","prima","pvrl2","sez6","tkrB","trop2","tyrp1","tyrp2","vgscb1","vgscb2","vgscb3","vgscb4","vldlr")

In [4]:
mouse_mart <- useEnsembl("ensembl", dataset = "mmusculus_gene_ensembl")

In [5]:
# Get UniProt IDs for these proteins
uniprot_mouse <- getBM(attributes = c("uniprotswissprot", "mgi_symbol"),
                      filters = "external_gene_name",
                      values = mouselist,
                      mart = mouse_mart)


In [6]:
mouse_ids <- uniprot_mouse$uniprotswissprot

In [7]:
uniprot_mouse <- getBM(attributes = c("uniprotswissprot", "mgi_symbol"),
                      filters = "uniprotswissprot",
                      values = mouse_ids,
                      mart = mouse_mart)


In [8]:
mouse_genes <- uniprot_mouse$mgi_symbol

In [9]:
mapping <- read.delim("HMD_HumanPhenotype.rpt", header = FALSE)

In [10]:
colnames(mapping) <- c("Human_Gene","hum_num", "Mouse_Gene", "Mouse_MGI_ID", "HPO_Term","last")

In [11]:
result <- mapping %>%
  filter(Mouse_Gene %in% mouse_genes) %>%
  select(Mouse_Gene, Human_Gene)

In [12]:
converted_mouse <- result$Human_Gene

In [13]:
ensembl <- useEnsembl("ensembl", dataset = "hsapiens_gene_ensembl")


In [14]:
# Fetches hgnc_symbol and unirpot ID
results_hum <- getBM(
  attributes = c("hgnc_symbol","uniprotswissprot"), # Attributes: UniProt ID and Gene Symbol
  filters = "hgnc_symbol",                      # Filter by UniProt IDs
  values = humanlist,                                 # Input your list of UniProt IDs
  mart = ensembl                                      # Use the Ensembl mart connection
)

In [15]:
human_genes <- results_hum$hgnc_symbol

In [16]:
human_genes <-c(human_genes,mouse_genes)

In [17]:
# Retriving module 1 from mic-3 since its the only microglia cell state that assigned PSEN1 to a module

In [18]:
modules <- read.csv("microglia_moduleTables/microstate3_modules.csv")
modules <- subset(modules, module == "micro1")

In [19]:
# Extracting the genes assigned to the same module PSEN1 is in

In [20]:
module_genes <- modules$gene_name

In [35]:
module_genes

[1] "AL669831.5"      "INTS11"          "SLC35E2B"        "SLC35E2A"       
   [5] "NADK"            "CFAP74"          "MORN1"           "TNFRSF14"       
   [9] "CEP104"          "KCNAB2"          "RERE"            "H6PD"           
  [13] "PIK3CD"          "CLSTN1"          "UBE4B"           "KIF1B"          
  [17] "PGD"             "AGTRAP"          "MTHFR"           "VPS13D"         
  [21] "DHRS3"           "KAZN"            "TMEM51"          "EFHD2"          
  [25] "DNAJC16"         "DDI2"            "FBXO42"          "PADI2"          
  [29] "RCC2"            "IGSF21"          "UBR4"            "CAPZB"          
  [33] "USP48"           "KDM1A"           "AL031428.1"      "HMGCL"          
  [37] "MACO1"           "ARID1A"          "WDTC1"           "WASF2"          
  [41] "THEMIS2"         "RPA2"            "RCC1"            "TRNAU1AP"       
  [45] "SNHG12"          "LAPTM5"          "ZCCHC17"         "BSDC1"          
  [49] "ZBTB8OS"         "S100PBP"         "PHC2"            "ZMYM4"          
  [53] "KIAA0319L"       "AGO1"            "AL139260.1"      "MACF1"          
  [57] "PABPC4"          "PPIE"            "TRIT1"           "CAP1"           
  [61] "SMAP2"           "P3H1"            "RNF220"          "TESK2"          
  [65] "GPBP1L1"         "ATPAF1"          "CMPK1"           "FAF1"           
  [69] "EPS15"           "OSBPL9"          "BTF3L4"          "CC2D1B"         
  [73] "TUT4"            "SHISAL2A"        "ACOT11"          "USP24"          
  [77] "FGGY"            "NFIA"            "TM2D1"           "DOCK7"          
  [81] "ATG4C"           "PGM1"            "SRSF11"          "ST6GALNAC3"     
  [85] "USP33"           "IFI44"           "TTLL7"           "PRKACB"         
  [89] "CTBS"            "ODF2L"           "LMO4"            "PKN2"           
  [93] "GBP2"            "LRRC8C-DT"       "RPAP2"           "EVI5"           
  [97] "FAM69A"          "CCDC18"          "DR1"             "FNBP1L"         
 [101] "BCAR3"           "DPYD"            "AGL"             "CDC14A"         
 [105] "AC093157.1"      "RNPC3"           "STXBP3"          "WDR47"          
 [109] "PSRC1"           "SORT1"           "AMPD2"           "GSTM4"          
 [113] "GSTM3"           "AHCYL1"          "C1orf162"        "TMIGD3"         
 [117] "RAP1A"           "CTTNBP2NL"       "WNT2B"           "AL390729.1"     
 [121] "SLC16A1-AS1"     "LRIG2"           "PHTF1"           "RSBN1"          
 [125] "HIPK1"           "TRIM33"          "GDAP2"           "NBPF26"         
 [129] "LINC00623"       "AC244453.3"      "SRGAP2C"         "AC244021.1"     
 [133] "NBPF15"          "SRGAP2B"         "RNF115"          "LIX1L"          
 [137] "TXNIP"           "NBPF12"          "PRKAB2"          "ACP6"           
 [141] "NBPF14"          "NBPF9"           "AC245297.3"      "NBPF19"         
 [145] "HIST2H2BE"       "VPS45"           "PLEKHO1"         "PRPF3"          
 [149] "RPRD2"           "MCL1"            "ENSA"            "ARNT"           
 [153] "PI4KB"           "RFX5"            "SNX27"           "GATAD2B"        
 [157] "CRTC2"           "IL6R"            "MEF2D"           "HDGF"           
 [161] "SLAMF8"          "DCAF8"           "F11R"            "USF1"           
 [165] "ATF6"            "OLFML2B"         "NOS1AP"          "PBX1"           
 [169] "POU2F1"          "CREG1"           "RCSD1"           "MPZL1"          
 [173] "DCAF6"           "SFT2D2"          "TBX19"           "VAMP4"          
 [177] "AL645568.1"      "DARS2"           "ZBTB37"          "RC3H1"          
 [181] "RABGAP1L"        "KIAA0040"        "COP1"            "RASAL2"         
 [185] "RALGPS2"         "SOAT1"           "CEP350"          "XPR1"           
 [189] "STX6"            "NPL"             "TSEN15"          "EDEM3"          
 [193] "TRMT1L"          "SWT1"            "PLA2G4A"         "ERVMER61-1"     
 [197] "RGS18"           "AL390957.1"      "RGS1"            "DENND1B"        
 [201] "NEK7"            "PTPRC"           "MIR181A1H

In [21]:
ad_genes <- read_excel("AD_genes.xlsx", col_names = FALSE)

New names:
• `` -> `...1`


In [22]:
ad_genes <- ad_genes$"...1"

In [33]:
gene_sets <- list(
  "AD Genes" = ad_genes,
  "Module Genes" = module_genes,
  "PSEN1 substrates" = human_genes
)


In [38]:

v <- venn.diagram(gene_sets,
                  fill = c("orange", "blue"),
                  alpha = c(0.5, 0.5), cat.cex = 1.5, cex=1.5,
                  filename=NULL)

ERROR: Error in venn.diagram(gene_sets, fill = c("orange", "blue"), alpha = c(0.5, : could not find function "venn.diagram"


tibble [7 × 5] (S3: tbl_df/tbl/data.frame)
 $ id   : chr [1:7] "1" "2" "3" "1/2" ...
 $ name : chr [1:7] "AD Genes" "Module Genes" "PSEN1 substrates" "AD Genes/Module Genes" ...
 $ item :List of 7
  ..$ : chr [1:1049] "FAM171A1" "NUDT5" "PROSER2" "UPF2" ...
  ..$ : chr [1:2321] "AL669831.5" "INTS11" "SLC35E2B" "SLC35E2A" ...
  ..$ : chr [1:50] "LRP6" "KCNE1" "IFNAR2" "KCNE2" ...
  ..$ : chr [1:146] "SEC61A2" "USP6NL" "CELF2" "DHTKD1" ...
  ..$ : chr [1:5] "SEZ6L2" "APP" "TREM2" "EPHB4" ...
  ..$ : chr [1:8] "IL6R" "F11R" "CD99" "PLXDC2" ...
  ..$ : chr "ACP2"
 $ count: int [1:7] 1049 2321 50 146 5 8 1
 $ label: chr [1:7] "1049" "2321" "50" "146" ...
